In [10]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from activ.pipeline import score_clusters
import matplotlib.pyplot as plt
import scipy
import h5py
from scipy.stats import entropy
from scipy.spatial.distance import pdist, squareform
import scipy.cluster.hierarchy as _sch
from activ import load_data
import pandas as pd
from math import ceil
from matplotlib import cm

In [43]:
fulldata = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/preprocessed_data_matrix.csv', encoding = "ISO-8859-1")
fulldata.shape

(586, 919)

In [12]:
patients = open('/Users/ahyeon/Desktop/TBIanalysis/data/patient_id.txt').read().split()

In [34]:
outcome = open('/Users/ahyeon/Desktop/TBIanalysis/data/data_matrix_subset_outcome.txt').read()

In [13]:
fulldata.rename( columns={'Unnamed: 0':'id'}, inplace=True )

In [39]:
data = load_data()
nmfdata = load_data(uoinmf=True)
oc_features = data.outcome_features
bm_features = data.biomarker_features

In [42]:
data.outcomes

array([[1. , 2. , 1. , ..., 0. , 2.3, 0.5],
       [1. , 2. , 1. , ..., 1. , 2. , 0.5],
       [1. , 2. , 1. , ..., 0.5, 4. , 1. ],
       ...,
       [1. , 1. , 2. , ..., 0. , 4. , 1. ],
       [1. , 2. , 1. , ..., 0.5, 4. , 1.5],
       [1. , 2. , 1. , ..., 0. , 4. , 1. ]])

In [44]:
fulldata.

SyntaxError: invalid syntax (<ipython-input-44-9d33caeb1b3e>, line 1)

In [40]:
oc_features

array(['GOSE_NeedAssistance3M', 'GOSE_WorkResume3M',
       'GOSE_DisruptRelation3M', 'GOSE_OtherIssues3M',
       'GOSE_OverallScore3M', 'NeuroPhysHeadache3mo',
       'NeuroPhysBalanceProbl3mo', 'NeuroPhysDizziness3mo',
       'NeuroPhysVisualProbl3mo', 'NeuroPhysFatigue3mo',
       'NeuroPhysLightSensitivity3mo', 'NeuroPhysNoiseSensitivity3mo',
       'NeuroPhysNumbnessTingling3mo', 'NeuroSleepDrowsiness3mo',
       'NeuroSleepSleepingLess3mo', 'NeuroSleepSleepingMore3mo',
       'NeuroSleepTroubleFallingAsleep3mo', 'NeuroCognitiveFoggy3mo',
       'NeuroCognitiveSlowedDown3mo',
       'NeuroCognitiveDiffConcentrating3mo',
       'NeuroCognitiveDiffRemembering3mo',
       'NeuroEmotionalMoreEmotional3mo', 'NeuroEmotionalNervousness3mo',
       'NeuroOverallRating3mo', 'GOSE_Reponder6M_Patient.alone',
       'GOSE_Reponder6M_Patient.plus.relative', 'GOSE_NeedAssistance6M',
       'GOSE_Travel6M', 'GOSE_SocialResume6M', 'GOSE_OtherIssues6M',
       'GOSE_Epilepsy6M', 'GOSE_EpilepsyRis

In [8]:
len(bm_features)

235

In [9]:
len(list(set(fulldatafeatures).intersection(bm_features)))

111

In [17]:
len(patients)

247

In [20]:
fulldata.columns

Index(['id', 'Age', 'Sex', 'RaceAmericanIndian', 'RaceAlaskan', 'RaceAmAl',
       'RaceAsian', 'RaceBlack', 'RaceHAPI', 'RaceWhite',
       ...
       'ICHponsright', 'ICHponsleft', 'ICHmedullaright', 'ICHmedullaleft',
       'ICHcerebellumright', 'ICHcerebellumleft', 'MidlineShiftmm',
       'cisternalcompressionSeverity', 'Fourthventricleshift',
       'Thirdventricleobliteration'],
      dtype='object', length=919)

In [29]:
fulldata[:3]

,id,Age,Sex,RaceAmericanIndian,RaceAlaskan,RaceAmAl,RaceAsian,RaceBlack,RaceHAPI,RaceWhite,...,ICHponsright,ICHponsleft,ICHmedullaright,ICHmedullaleft,ICHcerebellumright,ICHcerebellumleft,MidlineShiftmm,cisternalcompressionSeverity,Fourthventricleshift,Thirdventricleobliteration
0,BR-1001,18,Male,No,No,No,No,No,No,Yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BR-1003,49,Male,No,No,No,No,No,No,Yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BR-1004,24,Male,No,No,No,No,Yes,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
patients = []
for r in range(fulldata.shape[0]):
    for o,feat in enumerate(oc_features):
        if fulldata.feat != 'NA':
            patients.append(r)

In [38]:
len(np.unique(patients))

586

In [41]:
for ii,x in enumerate(fulldata.columns):
    if x == 'GOSE_NeedAssistance3M':
        print(ii)

332


In [31]:
# features with NA > 500 
fulldata.columns[fulldata.isnull().sum() > 500]

Index(['SesPrimChild', 'SesEduNoFather', 'SesEduNoMother', 'SesEduTypeFather',
       'SesEduTypeMother', 'InjSafetyAirbag', 'InjSafetySeatbelt',
       'GcsEDArrPupilReactL', 'EDOsmo', 'NeuroProcedureNumbers', 'ICUReadmit',
       'NumInj_to_AbdPelvis', 'DeathTimeSinceInj', 'DeathCause',
       'DeathCauseOther', 'GOSE_NeedFreqHelp3M', 'GOSE_AssistanceBaseline3M',
       'GOSE_NeedFreqHelp6M', 'GOSE_AssistanceBaseline6M',
       'GOSE_NeedFreqHelp12M', 'GOSE_AssistanceBaseline12M',
       'GOSE_WorkRestriction12M', 'GOSE_WorkBaseline12M',
       'GOSE_SocialRestriction12M', 'GOSE_SocialBaseline12M',
       'GOSE_DisrupExtent12M', 'GOSE_DisruptBaseline12M'],
      dtype='object')

In [37]:
# features with NA > 300 
list(fulldata.columns[fulldata.isnull().sum() > 200])

['SesPrimChild',
 'SesEduNoFather',
 'SesEduNoMother',
 'SesEduTypeFather',
 'SesEduTypeMother',
 'TBILoc',
 'TBILocMemoryGap',
 'InjSafetyAirbag',
 'InjSafetySeatbelt',
 'PresFHospTimeSinceInj',
 'EDArrTemp',
 'GcsEDPupilSizeR',
 'GcsEDArrPupilReactR',
 'GcsEDPupilSizeL',
 'GcsEDArrPupilReactL',
 'EDDischTemp',
 'GcsEDDischPupils',
 'EDOsmo',
 'EDaPtt',
 'EDCO2',
 'EDBunSI',
 'bldsampleTimePostInj',
 'bldsampleMinutesPostInj',
 'GFAP_24H',
 'UCHL1_24H',
 'GFAPAutoAb',
 'T_Tau',
 'P_Tau',
 'P_T_TauRatio',
 'X.4plexGFAP',
 'X.4plexNFL',
 'X.4plexTau',
 'X.4plexUCHL1',
 'X.1plexNFL',
 'pTau181pgmL',
 'nrgn_fred',
 'bdnf_fred',
 'icam_fred',
 'NewGFAP_SOFIA',
 'ED_ETOH',
 'rs6265',
 'rs6277',
 'rs6311',
 'rs17759659',
 'rs3219119',
 'rs11604671',
 'rs4938016',
 'NeuroProcedureNumbers',
 'OtherProcedureInHospital',
 'Numberofproceduresforothersurgeries',
 'ICUAdmTimeSinceInj',
 'ICUDischTimeSinceInj',
 'ICUReadmit',
 'NumInj_to_AbdPelvis',
 'NumInj_to_ChestThorax',
 'NumInj_to_External',
 

In [17]:
[(i, x) for i, x in enumerate(oc_features) if 'GOSE_Overall' in x]

[(4, 'GOSE_OverallScore3M'), (32, 'GOSE_OverallScore6M')]

In [40]:
oc_features

array(['GOSE_NeedAssistance3M', 'GOSE_WorkResume3M',
       'GOSE_DisruptRelation3M', 'GOSE_OtherIssues3M',
       'GOSE_OverallScore3M', 'NeuroPhysHeadache3mo',
       'NeuroPhysBalanceProbl3mo', 'NeuroPhysDizziness3mo',
       'NeuroPhysVisualProbl3mo', 'NeuroPhysFatigue3mo',
       'NeuroPhysLightSensitivity3mo', 'NeuroPhysNoiseSensitivity3mo',
       'NeuroPhysNumbnessTingling3mo', 'NeuroSleepDrowsiness3mo',
       'NeuroSleepSleepingLess3mo', 'NeuroSleepSleepingMore3mo',
       'NeuroSleepTroubleFallingAsleep3mo', 'NeuroCognitiveFoggy3mo',
       'NeuroCognitiveSlowedDown3mo',
       'NeuroCognitiveDiffConcentrating3mo',
       'NeuroCognitiveDiffRemembering3mo',
       'NeuroEmotionalMoreEmotional3mo', 'NeuroEmotionalNervousness3mo',
       'NeuroOverallRating3mo', 'GOSE_Reponder6M_Patient.alone',
       'GOSE_Reponder6M_Patient.plus.relative', 'GOSE_NeedAssistance6M',
       'GOSE_Travel6M', 'GOSE_SocialResume6M', 'GOSE_OtherIssues6M',
       'GOSE_Epilepsy6M', 'GOSE_EpilepsyRis

In [21]:
data.outcomes[:,0]

array([1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 2., 1., 2., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1.

In [8]:
data.outcomes[:,0]

array([1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 2., 1., 2., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1.